# V-rep Demo for Poppy-Control in Simulator
It's a demo introduce how to control Poppy-Torso in simulator (V-rep).Please make sure:
+ 1. You have correctly installed V-rep and have a python 2.7 distribution. If not, please check README.md.
+ 2. Your have all the libs need, if you are not sure about which libs to use, a pre-packaged Python distribution called Anaconda is strongly recommended here. To get there please see the following below: https://www.continuum.io/downloads
+ 3. Or you can use your own python distribution and install them by use "pip install" in terminal. Like "pip install pypot poppy_humanoid"  
(Or replace it with poppy_torso if you use Torso)

# Initialization Check
To check if everything is correctly installed, you can run the following code. It shoud run without raising an error, when everything is probably installed. What we do here is to initialize the poppy enviroment in V-rep.  

Tipps:  
+ You can run IPython Notebook code cells by selecting them and clicking the play button or by pressing shift+enter.

In [2]:
from poppy.creatures import PoppyTorso
import numpy as np
import time
import math

poppy = PoppyTorso(simulator='vrep')

# Motor Retrieve
+ Firstly, with the following code, you can retrieve the list of all available motors.   
+ The motors variable contains the list of all motors attached to the current robot.  
+ By default, each motor prints its name, its id, and its current position:

In [3]:
poppy.motors

[<DxlMotor name=l_elbow_y id=44 pos=0.0>,
 <DxlMotor name=head_y id=37 pos=-1.8>,
 <DxlMotor name=r_arm_z id=53 pos=-0.0>,
 <DxlMotor name=head_z id=36 pos=0.0>,
 <DxlMotor name=r_shoulder_x id=52 pos=0.0>,
 <DxlMotor name=r_shoulder_y id=51 pos=0.8>,
 <DxlMotor name=r_elbow_y id=54 pos=0.0>,
 <DxlMotor name=l_arm_z id=43 pos=0.0>,
 <DxlMotor name=abs_z id=33 pos=-0.0>,
 <DxlMotor name=bust_y id=34 pos=0.2>,
 <DxlMotor name=bust_x id=35 pos=0.0>,
 <DxlMotor name=l_shoulder_x id=42 pos=-1.5>,
 <DxlMotor name=l_shoulder_y id=41 pos=0.0>]

# Control Motor

You can access a specific motor directly using its name:

In [10]:
poppy.head_z

<DxlMotor name=head_z id=36 pos=0.0>

You can directly get the current angle of a motor by using:

In [ ]:
print poppy.head_y.present_position

You can directly access all the motors from the torso using the torso alias.  
To retrieve the list of motors alias available using:

In [15]:
poppy.alias

[u'head', u'r_arm', u'torso', u'l_arm', u'arms']

Each alias contains a list of motors.  
To creat a dictionary of motor alias using:

In [16]:
{m.name: m.present_position for m in poppy.head}

{u'head_y': 0.0, u'head_z': 0.0}

To set motor to a specific angle using:

In [13]:
poppy.head_z.goal_position = 0
poppy.head_y.goal_position = 0


Note: While the full list of motor registers is available, not all of them are having an effect in the V-REP simulation. For instance, modifying the pid of a motor won't affect the simulation.

Currently in the V-REP simulator you can use:
+ present_position (R): the actual position of the motor (usually from -180° to 180°)
+ goal_position (RW): the target position of the motor, that is to say the position it will try to reach (same range and units than the present position)
+ present_load (R): the current load applied on the motor (expressed in % of the max supported load)
+ torque_limit (RW): the maximum torque that a motor can applied (also expressed in % of the max supported load)
+ compliant (RW): whether the motor is compliant: if it resits or not when manually turned
+ angle_limit (R): the position limits (lower and upper) of the motor. Some motors are restrained to a smaller position range to avoid breaking other parts.
Support for additional features may be added in future version.

# Object Interaction

Reset the simulator

In [3]:
poppy.reset_simulation()

Set up a controller

In [4]:
io = poppy._controllers[0].io

Add a object in simulator

In [5]:
name = 'cube'
position = [0, -0.15, 0.8] # X, Y, Z
sizes = [0.01, 0.01, 0.01] # in meters
mass = 0 # in kg
io.add_cube(name, position, sizes, mass)



# Pseudo-CV algorithm

In [6]:
# Output the heading direction of camera

def headForwardDirection():
    angleNegativeY = poppy.head_z.present_position
    angleSurfaceXY = - poppy.head_y.present_position

    angleNegativeY = angleNegativeY / 180 * 3.14159
    angleSurfaceXY = angleSurfaceXY / 180 * 3.14159

    y = - np.cos(angleSurfaceXY) * np.cos(angleNegativeY)
    x = np.cos(angleSurfaceXY) * np.sin(angleNegativeY)
    z = np.sin(angleSurfaceXY)

    forwardDire = [x, y, z]
    return forwardDire

# Output the relative position of object w.r.t camera position

def objectRelPosition(name):
    objectPos = io.get_object_position(name)
    positionCameraOri = [0, -0.05, 1.06] # Camera's position in Vrep

    objectRelPos = [objectPos[i] - positionCameraOri[i] for i in xrange(3)]

    return objectRelPos

# Output the angels(horizontal, vertical) of object w.r.t the center of camera sight
# If object is not in sight, return False

def canSeeJudge(headForwardDirection, objectRelPos):
    orthognalBasis1 = headForwardDirection
    orthognalBasis2 = [orthognalBasis1[1], -orthognalBasis1[0], 0]
    normOrthBasis2 = np.linalg.norm(orthognalBasis2)
    orthognalBasis2 =  [orthognalBasis2[i] / normOrthBasis2 for i in xrange(3) ]
    orthognalBasis3 = np.cross(orthognalBasis2, orthognalBasis1)

    objectProjectionOnOrthBasis1 = np.dot(objectRelPos, orthognalBasis1)
    if objectProjectionOnOrthBasis1 < 0:
        return False
    objectProjectionOnOrthBasis2 = np.dot(objectRelPos, orthognalBasis2)
    objectProjectionOnOrthBasis3 = np.dot(objectRelPos, orthognalBasis3)

    newCoordinate = [objectProjectionOnOrthBasis1, objectProjectionOnOrthBasis2, objectProjectionOnOrthBasis3]

    tt = [1, 0, 0]

    t = [objectProjectionOnOrthBasis1, objectProjectionOnOrthBasis2, 0]
    angle1 = np.arccos(np.dot(tt, t) / np.linalg.norm(t)) / 3.14159 * 180

    if abs(angle1) > 37:
        return False
    t = [objectProjectionOnOrthBasis1, 0, objectProjectionOnOrthBasis3]
    angle2 = np.arccos(np.dot(tt, t) / np.linalg.norm(t)) / 3.14159 * 180
    if abs(angle2) > 18.5:
        return False
    
    if objectProjectionOnOrthBasis3 < 0 and angle2 > 0:
        angle2 = -angle2

    if objectProjectionOnOrthBasis2 > 0 and angle1 > 0:
        angle1 = -angle1
    return angle1, angle2

# Output the state of problem
def getState(angle, m, n):
    if not angle:
        print "The object can't be seen"
        return False
    angle1, angle2 = angle
    state1 = math.ceil(abs(np.sin(angle1 / 180.0 * 3.14159) / np.sin(37 / 180.0 * 3.14159) * m))
    state2 = math.ceil(abs(np.sin(angle2 / 180.0 * 3.14159) / np.sin(18.5 / 180.0 * 3.14159) * m))
    if angle1 > 0:
        state1 = -state1
    if angle2 < 0:
        state2 = -state2
    return (state1, state2)

The object can't be seen
False


# Example of Using PseudoCV Algorithm
Perform expected circle trace with object

In [ ]:
# Draw circle and print the state
t = time.time()
while 1:
    x = np.sin(2 * 3.14159 * time.time() * 0.1) * 0.5
    z = np.cos(2 * 3.14159 * time.time() * 0.1) * 0.5
    io.set_object_position('cube', position=[x, -2, z + 1])
 
    if time.time() - t > 0.1:
        print getState(canSeeJudge(headForwardDirection(), objectRelPosition(name)), 10, 10)
        t = time.time()

Perform single line trace with object

In [ ]:
# Draw single line trace and print the state
t = time.time()
while 1:
    x = np.sin(2 * 3.14159 * time.time() * 0.1)
    io.set_object_position('cube', position=[x, -1, 1])

    if time.time() - t > 0.1:
        print getState(canSeeJudge(headForwardDirection(), objectRelPosition(name)), 10, 10)
        t = time.time()